In [1]:
Pkg.update() 
Pkg.add("JuMP")
Pkg.add("GLPKMathProgInterface")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: Package JuMP is already installed
INFO: Package GLPKMathProgInterface is already installed
INFO: No packages to install, update or remove


In [2]:
using JuMP
using MathProgBase
using GLPKMathProgInterface

In [3]:
include("util.jl")
include("network.jl")

In [4]:
# read in neural networks
large_nnet = read_nnet("ACASXU_nnet_1.txt")
small_nnet = read_nnet("small_nnet.txt")

Network(Layer[Layer([1.5, 1.5], [1.0; 1.0]), Layer([2.5, 2.5], [2.0 2.0; 2.0 2.0]), Layer([3.5], [3.0 3.0])])

In [7]:
function add_input_constraints(nnet::Network, m::Model, upperBounds::Vector{Float64}, lowerBounds::Vector{Float64}, 
                               A::Matrix{Float64} = [eye(length(upperBounds)); -eye(length(lowerBounds))])
    n_inputs = size(nnet.layers[1].weights)[2]
    b = [upperBounds; -lowerBounds]
    x_in = @variable(m, [1:n_inputs])
    @constraint(m, A*x_in .<= b)
    return x_in
end

add_input_constraints (generic function with 2 methods)

In [8]:
function add_output_constraints(nnet::Network, m::Model, upperBounds::Vector{Float64}, lowerBounds::Vector{Float64}, 
                                A::Matrix{Float64}=[eye(length(upperBounds)); -eye(length(lowerBounds))])
    n_outputs = size(nnet.layers[length(nnet.layers)].weights)[1]
    b = [upperBounds; -lowerBounds]
    x_out = @variable(m, [1:n_outputs])
    @constraint(m, A * x_out .<= b)
    return x_out
end

add_output_constraints (generic function with 2 methods)

In [11]:
function add_network_constraints(nnet::Network, m::Model,  x_in::Array{JuMP.Variable,1}, x_out::Array{JuMP.Variable,1})
    layers = nnet.layers
    M = 1 # define reasonable M?
    
    x_net = @variable(m, [i=1:length(layers), j=1:length(layers[i].bias)]) 
    deltas = @variable(m, [i=1:length(layers), j=1:length(layers[i].bias)]) 

    for i in 1:length(layers) - 1
        weights = layers[i].weights
        bias = layers[i].bias
        
        for j in 2:length(bias)
            dot_prod = 1
            if i == 1
                dot_prod = @expression(m, [k=1:size(nnet.layers[1].weights)[2]], sum(weights[j,k] * x_in[k]))
            else
                dot_prod = @expression(m, [k=1:length(layers[i-1].bias)], sum(weights[j,k] * x_net[i-1,k]))
            end
            @constraint(m, x_net[i,j] .>= dot_prod + bias[j]) 
            @constraint(m, x_net[i,j] .<= dot_prod + bias[j] + M*deltas[i,j])
            @constraint(m, x_net[i,j] >= 0)
            @constraint(m, x_net[i,j] <= M*(1 - deltas[i,j]))
        end
    end
    
    i = length(layers)
    weights = layers[i].weights
    bias = layers[i].bias
    for l in 1:length(bias)
        dot_prod = @expression(m, [k=1:length(nnet.layers[i-1].bias)], sum(weights[l,k] * x_net[l,k]))
        @constraint(m, x_out[l] .>= dot_prod + bias[l])
        @constraint(m, x_out[l] .<= dot_prod + bias[l] + M*deltas[i,l])
        @constraint(m, x_out[l] >= 0)
        @constraint(m, x_out[l] <= M*(1 - deltas[i,l]))
    end
end

add_network_constraints (generic function with 1 method)

In [12]:
# output for 0.0 = 54.5

In [13]:
# should return optimal
test_model = Model(solver = GLPKSolverLP())
input_lower_bounds = [0.0]
input_upper_bounds = [0.0]
x_in = add_input_constraints(small_nnet, test_model, input_upper_bounds, input_lower_bounds)
output_lower_bounds = [54.5]
output_upper_bounds = [54.5]
x_out = add_output_constraints(small_nnet, test_model, output_upper_bounds, output_lower_bounds)
add_network_constraints(small_nnet, test_model, x_in, x_out)
status = solve(test_model)
print(status)

Infeasible

In [14]:
# should return infeasible
test_model = Model(solver = GLPKSolverLP())
input_lower_bounds = [0.0]
input_upper_bounds = [0.0]
x_in = add_input_constraints(small_nnet, test_model, input_upper_bounds, input_lower_bounds)
output_lower_bounds = [0.0]
output_upper_bounds = [54.0]
x_out = add_output_constraints(small_nnet, test_model, output_upper_bounds, output_lower_bounds)
add_network_constraints(small_nnet, test_model, x_in, x_out)
status = solve(test_model)
print(status)

Infeasible

In [76]:
print(small_nnet.layers)

Layer[Layer([1.5, 1.5], [1.0; 1.0]), Layer([2.5, 2.5], [2.0 2.0; 2.0 2.0]), Layer([3.5], [3.0 3.0])]

In [ ]:
# Setup our model and test on small_nnet
model = Model(solver = GLPKSolverLP()) # use GLPK LP as our solver

input_lower_bounds = [0.0]
input_upper_bounds = [0.0]
x_in = add_input_constraints(small_nnet, input_lower_bounds, input_upper_bounds, model)

output_lower_bounds = [1.0]
output_upper_bounds = [54.5]
x_out = add_output_constraints(small_nnet, output_lower_bounds, output_upper_bounds, model)

#add_network_constraints(small_nnet, model, x_in, x_out)

In [ ]:
print(small_nnet.layers)

In [ ]:
# Solve the model
status = solve(model)
print(model)
print(status)